In [5]:
%load_ext autoreload
%autoreload 2

In [1]:
pip install --upgrade pyldpc

  Created wheel for pyldpc: filename=pyldpc-0.7.9-py3-none-any.whl size=14307 sha256=a1a31d20cf86600534e1abcc5e24b408cf47fc5bdc917056cb2ae61169ada33d
  Stored in directory: c:\users\myros\appdata\local\pip\cache\wheels\00\f1\5f\d89afe3e8ba95547a385c49f048742fb3fd0aab91eb274c490
Successfully built pyldpc
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
from pyldpc import make_ldpc, encode, decode, get_message

n = 15
d_v = 4
d_c = 5
snr = 200
H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
k = G.shape[1]
v = np.random.randint(2, size=k)
y = encode(G, v, snr)
d = decode(H, y, snr)
x = get_message(G, d)
assert abs(x - v).sum() == 0

In [3]:
from decoder import bit_flip_dec

In [4]:
decoded_x = bit_flip_dec(H,y)

100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 21708.75it/s]


In [6]:
abs(x - decoded_x).sum() #comparison with pyldpc decoder

ValueError: operands could not be broadcast together with shapes (6,) (15,) 

In [8]:
abs(x - v).sum() #comparison with input msg

(15,)

In [14]:
y

array([-1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,  1.,  1.,
        1.,  1.])

In [12]:
v

array([0, 0, 1, 1, 1, 0])

In [24]:
np.arange(H.shape[0])[(H @ y) != 0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [18]:
y.shape

(15,)

In [21]:
(H @ y).shape

(12,)